In [ ]:
%reload_ext autoreload

%autoreload 2

In [ ]:
import os
import directlfq
import alphabase.tools.data_downloader

# Get the directory of the directlfq package
directlfq_dir = os.path.dirname(directlfq.__file__)

# Navigate up two levels to reach the project root
project_root = os.path.abspath(os.path.join(directlfq_dir, '..'))

output_dir = os.path.join(project_root, 'test_data', 'system_tests', "Spectronaut_LargeFC")

os.makedirs(output_dir, exist_ok=True)
# Now use the absolute path in the DataShareDownloader call
alphabase.tools.data_downloader.DataShareDownloader(url="https://datashare.biochem.mpg.de/s/T8sFUGuaQb0d1zP", output_dir=output_dir).download()


In [ ]:
INPUT_FILE = f"{output_dir}/re_run_SN15/20220607_153923_MP-LFC-MS1var-OT-S1-120kMS1_Report.tsv.zip"

DIRECTLFQ_FILE = f"{INPUT_FILE}.protein_intensities.tsv"

PROTEIN2ORGANISM_FILE = f"{output_dir}/re_run_SN15/organism_map.tsv"
SAMPLEMAP  = f"{output_dir}/re_run_SN15/samplemap.tsv"


In [ ]:
import directlfq.lfq_manager as lfqmgr

lfqmgr.run_lfq(input_file=INPUT_FILE)


In [ ]:
import directlfq.benchmarking as lfqbenchmark
import directlfq.utils as lfqutils


samples_used = lfqutils.get_samples_used_from_samplemap_file(SAMPLEMAP, cond1="S1", cond2="S2")


restable_directlfq = lfqbenchmark.ResultsTableDirectLFQ(input_file=DIRECTLFQ_FILE, input_name="directLFQ", samples_c1=samples_used[0], samples_c2=samples_used[1])


In [ ]:
organism_annotator_spectronaut_based = lfqbenchmark.OrganismAnnotatorSpectronaut(mapping_file=PROTEIN2ORGANISM_FILE,protein_column="PG.ProteinGroups", organism_column="PG.Organisms")
organism_annotator_spectronaut_based.annotate_table_with_organism(restable_directlfq)

In [ ]:
display(restable_directlfq.formated_dataframe["organism"].unique())

In [ ]:
import numpy as np
import directlfq.test_utils as testutils

organisms_to_plot = [ "Saccharomyces cerevisiae (strain ATCC 204508 / S288c)", "Homo sapiens", "Caenorhabditis elegans"]
fcs_to_expect = [  np.log2(2), 0, -np.log2(1.3)]

organism2expectedfc = { "Saccharomyces cerevisiae (strain ATCC 204508 / S288c)" : np.log2(2), "Homo sapiens": 0, "Caenorhabditis elegans": -np.log2(1.3)}
organism2CI95={"Caenorhabditis elegans" : 0.4, "Homo sapiens" :  0.3, "Saccharomyces cerevisiae (strain ATCC 204508 / S288c)" : 0.5}


testutils.RatioChecker(formatted_df=restable_directlfq.formated_dataframe, organism2expectedfc=organism2expectedfc, organism2CI95=organism2CI95)



In [ ]:
import seaborn as sns
import directlfq.visualizations
import directlfq.benchmarking
import matplotlib.pyplot as plt
import numpy as np


class MultiOrganismMultiMethodBoxPlotAdapted(directlfq.visualizations.MultiOrganismMultiMethodBoxPlot):
    def plot_boxplot(self):
        color_palette = sns.color_palette(self._colorlist_hex, n_colors=len(self._fcs_to_expect))
        sns.boxplot(data=self._method_ratio_results_table, x="method", y = "log2fc", hue= "organism", palette=color_palette, hue_order=self._organisms_to_plot, ax=self.ax, linewidth=0.8)



methodname2formatted_df = {"standard": restable_directlfq.formated_dataframe}

fig, axes = plt.subplots(1, 2,figsize=(4, 4.363))

merged_table = directlfq.benchmarking.ResultsTableMerger(method_name2results_df=methodname2formatted_df).merged_table


MultiOrganismMultiMethodBoxPlotAdapted(method_ratio_results_table=merged_table, ax = axes[0], organisms_to_plot=organisms_to_plot,fcs_to_expect= fcs_to_expect)



directlfq.visualizations.MultiOrganismIntensityFCPlotter(ax=axes[1], resultstable_w_ratios=restable_directlfq, organisms_to_plot = organisms_to_plot, fcs_to_expect= fcs_to_expect, title="")

